In [1]:
import utils
import os

In [8]:
## Import necessary library to load data
from llama_index.core import SimpleDirectoryReader
import os
from llama_index.readers.json import JSONReader
## Initialize file path from where the data is to be loaded
directory_path = "dataset/extracted_best/"
## load all the markdownfiles
markdown_files = SimpleDirectoryReader(directory_path).load_data()
## Initialize JSONreader to Load additional structured data
reader = JSONReader()
## Load the JSON
json_files = []
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):
        file_path = os.path.join(directory_path,filename)
        json_files.extend(reader.load_data(input_file=file_path,extra_info={}))
## Concat both Markdown and Json files, it does not matter for LLM
documents=markdown_files+json_files


In [9]:
## Extracting nodes/Chunk info from parent docs into smaller pieces 
from llama_index.core.node_parser import MarkdownNodeParser
node_parser=MarkdownNodeParser.from_defaults()
nodes=node_parser.get_nodes_from_documents(documents)

In [10]:
## Index the parsed nodes/ Convert it into vector embeddings
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.nomic import NomicEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
## Initialize LLM
Settings.llm=Ollama(model="llama3",temperature=0)
## Initialize the embedding models
Settings.embed_model=NomicEmbedding(model_name="nomic-embed-text-v1", inference_mode="local")
## Store the embeddings into Vectorstore
index = VectorStoreIndex(nodes)

In [11]:
## Create query engine for retrieval of data from vector db
query_engine = index.as_query_engine()  

In [12]:
## Define LLM, here using Ollama to run local model
llm=Ollama(model="llama3.1",temperature=0)

In [13]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.tools.types import ToolMetadata
## Define the tool and provide the necessary metadata for the LLM
query_engine_tools = [QueryEngineTool(
    query_engine=query_engine,
        metadata=ToolMetadata(
            name="retriever",
            description="A RAG engine with information about AMM, CMM, DMM and EMM connectors of Nicomatic manufacturers.")                          
        )]
## Convert this qeury tool from llama index into a langchain tool
llamaindex_to_langchain_converted_tools = [t.to_langchain_tool() for t in query_engine_tools]

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
## Web search
from langchain_community.tools.tavily_search import TavilySearchResults
search_tool = TavilySearchResults()
## transform this into a LLM usable tool
langchain_tools = [search_tool]
## concat both tool into a toolset
tools=llamaindex_to_langchain_converted_tools+ langchain_tools

In [16]:
## Define LLM
from langchain_community.chat_models import ChatOllama
llm=ChatOllama(model="llama3.1")

In [17]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain import hub
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
## Initialize a dictionary to hold the conversational memory
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [19]:
## Using custom prompt with ReAct capability for agents 
prompt = hub.pull("intern/test")
## Initialize react agents
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
## Initialize agent execture crafted to incorporate the conversation history
agent_executer = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executer,
    get_session_history,    
    input_messages_key="input",
    history_messages_key="chat_history")

c:\Users\s.vijaykumar\Desktop\skanda\bot\venv\Lib\site-packages\langsmith\client.py:5282: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [23]:
## Inferencing
while True:
    input1=input("enter here:\n")
    if input1.lower()=="done":
        break
    else:
        agent_with_chat_history.invoke(
            {"input": input1},
            config={
                "configurable": {"session_id": "abc123"}
            } )



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Action: None (as we don't need to use any tools in this caseInvalid Format: Missing 'Action Input:' after 'Action:'Let's try again.

Since you said "I dont know this too", let's proceed with another question. However, I'll make sure to follow the correct format.

Thought: Do I need to use a tool? No
Action: None (as we don't need to use any tools in this caseInvalid Format: Missing 'Action Input:' after 'Action:'Thought: Do I need to use a tool? No
Action: Ask another question
Action Input: What are the main differences between AMM, CMM, DMM, and EMM connectors that you think might affect your choiceAsk another question is not a valid tool, try one of [retriever, tavily_search_results_json].Thought: Do I need to use a tool? No
Action: None (as we don't need to use any tools in this caseInvalid Format: Missing 'Action Input:' after 'Action:'Let's try again.

Thought: Do I need to use a tool? No
Action: Ask anot